In [ ]:
import sys
sys.path.append('../..')
import torch                    
import torchvision
import pandas as pd
import torch
import numpy as np
import torchvision
from torch.utils.data import TensorDataset
from torchvision import transforms
from common_files.model_utils import tokenize_mask
from common_files.custom_sets import MemesDataset


print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)


This notebook creates tensor datasets for the Hateful Memes challenge. 

The dataset files can be downloaded here: https://www.kaggle.com/datasets/parthplc/facebook-hateful-meme-dataset
Since the test set does not have known labels, we will need to sample from the train set to create our own test set with known labels. 

In [6]:
data_path = ".../facebook_memes/data/"

In [ ]:
train = pd.read_json( data_path + 'train.jsonl', lines=True)
train.head()

In [ ]:
test = train.sample(1000)
test.reset_index(drop=True).to_csv(data_path + 'test.csv', index=False)

In [ ]:
train = train.drop(test.index).sample(frac =1) #dropping test and shuffling
train.reset_index(drop=True).to_csv(data_path + 'train.csv', index=False)

In [5]:
def save_data(data, split, typ):
    path = data_path + split + "_" + typ + "inputs.pt"
    torch.save(data, path)

In [ ]:
for split in ["train", "test", "val"]:
    if split == "val":
        df = pd.read_json(data_path + 'dev.jsonl', lines=True)
    else:
        df = pd.read_csv(data_path + split + '.csv')
        
    img_input = MemesDataset(
        data_frame= df,
        root_dir= data_path,
        transform=transforms.Compose([
            transforms.Resize((224,224)),
            transforms.ToTensor()
        ])
    )
    save_data(img_input, split, "img")
    
    sentences = df.text.values
    input_ids, att, labels = tokenize_mask(sentences, df["label"].values)
    txt_input = TensorDataset(input_ids, att, labels)
    save_data(txt_input, split, "txt")
    